<a href="https://colab.research.google.com/github/gabrielhozana/Oranges_vs_Grapefruit_Model_Using_Tensorflow/blob/main/Oranges_vs_Grapefruit_Model_Using_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [5]:
# https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb#scrollTo=hMY4CFezjcG-
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [6]:
# https://www.kaggle.com/joshmcadams/oranges-vs-grapefruit?select=citrus.csv
!kaggle datasets download -d joshmcadams/oranges-vs-grapefruit

  0% 0.00/61.2k [00:00<?, ?B/s]
100% 61.2k/61.2k [00:00<00:00, 22.8MB/s]


In [7]:
!unzip oranges-vs-grapefruit.zip

Archive:  oranges-vs-grapefruit.zip
  inflating: citrus.csv              


In [9]:
import pandas as pd
df = pd.read_csv("citrus.csv")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  object 
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 468.9+ KB


In [13]:
df.head(10000)

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


In [14]:
#ubah label
df.name[df.name == 'orange'] = 0
df.name[df.name == 'grapefruit'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
#dataset harus dalam bentuk array agar dapat diproses oleh model
dataset = df.values

In [16]:
dataset

array([[0, 2.96, 86.76, 172, 85, 2],
       [0, 3.91, 88.05, 166, 78, 3],
       [0, 4.42, 95.17, 156, 81, 2],
       ...,
       [1, 15.59, 256.5, 168, 82, 20],
       [1, 15.92, 260.14, 142, 72, 11],
       [1, 16.45, 261.51, 152, 74, 2]], dtype=object)

In [17]:
# pilih 4 kolom terakhir sebagai atribut
X = dataset[:,1:6]
# bilangan sebelum koma untuk memilih baris pada dataframe
# bilangan setelah koma untuk memilih kolom pada dataframe

In [18]:
y = dataset[:,0]

In [19]:
# Normalization
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.        , 0.        , 0.74025974, 0.63529412, 0.        ],
       [0.07042254, 0.00738197, 0.66233766, 0.55294118, 0.01851852],
       [0.10822832, 0.04812589, 0.53246753, 0.58823529, 0.        ],
       ...,
       [0.93624907, 0.97133047, 0.68831169, 0.6       , 0.33333333],
       [0.96071164, 0.99216023, 0.35064935, 0.48235294, 0.16666667],
       [1.        , 1.        , 0.48051948, 0.50588235, 0.        ]])

In [20]:
# pisahkan data training dan testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, y, test_size=0.3)

In [21]:
'''Karena tadi kita mengubah nilai-nilai pada kolom name menjadi bilangan numerik, yang artinya kita mengubah label menjadi tipe data boolean, 
maka kita perlu mengubah tipe data tersebut menjadi float32 dengan cara berikut.
'''
import numpy as np
     
Y_train = Y_train.astype(np.float32)
Y_test = Y_test.astype(np.float32)

In [22]:
from keras.models import Sequential
from keras.layers import Dense

In [24]:
X_train.shape

(7000, 5)

In [25]:
model = Sequential([
                    Dense(32, activation='relu', input_shape=(5,)),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid'),])

In [26]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
219/219 [==============================] - 15s 1ms/step - loss: 0.6774 - accuracy: 0.5165
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.6232 - accuracy: 0.7748
Epoch 3/100
219/219 [==============================] - 0s 1ms/step - loss: 0.5596 - accuracy: 0.8858
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4705 - accuracy: 0.9089
Epoch 5/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3804 - accuracy: 0.9194
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3031 - accuracy: 0.9286
Epoch 7/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2614 - accuracy: 0.9216
Epoch 8/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2301 - accuracy: 0.9248
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2143 - accuracy: 0.9239
Epoch 10/100
219/219 [==============================] - 0s 1ms/step - loss: 0.2012 - accur

In [28]:
model.evaluate(X_test, Y_test)
# elemen pertama adalah loss dan elemen kedua adalah akurasi

94/94 [==============================] - 1s 929us/step - loss: 0.1704 - accuracy: 0.9303


[0.1704494059085846, 0.9303333163261414]